# __mfusg_freyberg_example__

In [1]:
pip install pooch

In [11]:
from pathlib import Path
from pprint import pformat
# import git
import pooch
import flopy
import os
import shutil

In [13]:
org_d = os.path.join('freyberg_usg')
tmp_d = os.path.join('v01_Beta')                                                  # New folder
if os.path.exists(tmp_d):
    shutil.rmtree(tmp_d)
shutil.copytree(org_d,tmp_d)
# hbd.prep_bins(tmp_d)                                      # executables  
# hbd.prep_deps(tmp_d)   

'v01_Beta'

In [14]:
sim = flopy.mf6.MFSimulation.load(sim_ws=tmp_d)                   # Beta _____ to get freyberg.usg.nam

loading simulation...
  loading simulation name file...


MFDataException: An error occurred in package "None". The error occurred while loading package file in the "load" method.
Additional Information:
(1) File E:\15_REPOS\00_BETAMI\02_floPy__________________\01_mfusg_FREYBERG_example_______\v01_Beta\mfsim.nam of type nam could not be opened.

In [6]:
root_name = "freyberg.usg"                 

In [7]:
model_ws = freyberg_usg / root_name.replace(".", "_")

NameError: name 'freyberg_usg' is not defined

In [ ]:
from flopy.discretization import UnstructuredGrid                                     # dependency
mfgrid = UnstructuredGrid.from_gridspec(str(model_ws / f"{root_name}.gsf"))

In [ ]:
from flopy.utils.geometry import LineString

lines = [
    LineString(ls)
    for ls in [
        [(623000, 3364000), (623000, 3372000)],
        [(623650, 3364000), (623650, 3372000)],
    ]
]

In [ ]:
gwf = flopy.mfusg.MfUsg.load(
    f"{root_name}.nam",
    model_ws=str(model_ws),
    verbose=False,
    check=False,
    exe_name="mfusg",
) # Loading the model
bas6 = gwf.get_package("bas6")
ibound = bas6.ibound.array

In [ ]:
from tempfile import TemporaryDirectory

temp_dir = TemporaryDirectory()
work_dir = Path(temp_dir.name) / "freyberg_usg"

gwf.change_model_ws(str(work_dir))
gwf.write_name_file()
gwf.write_input()
success, buff = gwf.run_model(silent=True, report=True)
assert success, pformat(buff)

In [ ]:
levels = np.arange(30, 35.4, 0.1)
fig = plt.figure(figsize=(15, 10))

for layer, h in enumerate(head):
    ax = fig.add_subplot(1, len(head), layer + 1)
    ax.set_title(f"Freyberg head (layer {layer})")
    pmv = flopy.plot.PlotMapView(modelgrid=mfgrid, ax=ax)
    mesh = pmv.plot_array(h, alpha=0.2)
    grid = pmv.plot_grid(alpha=0.2)
    shps = pmv.plot_shapes(lines, edgecolor="purple", lw=2, alpha=0.8)
    inac = pmv.plot_inactive(ibound=ibound)
    ctrs = pmv.contour_array(h, levels=levels)

In [ ]:

# -

# A `head` argument can be provided to `CrossSectionPlot.contour_array()` to show the phreatic surface.

# +
fig = plt.figure(figsize=(25, 5))

for i, line in enumerate(lines):
    ax = fig.add_subplot(1, len(lines), i + 1)
    ax.set_title(f"Freyberg head cross-section (line {i})")
    xsect = flopy.plot.PlotCrossSection(
        modelgrid=mfgrid, ax=ax, line={"line": lines[i]}, geographic_coords=True
    )
    xsect.plot_array(head, head=head, alpha=0.4)
    xsect.plot_ibound(ibound=ibound, head=head)
    xsect.plot_inactive(ibound=ibound)
    contours = xsect.contour_array(
        head, masked_values=[999.0], head=head, levels=levels, alpha=1.0, colors="blue"
    )
    plt.clabel(contours, fmt="%.0f", colors="blue", fontsize=12)
    xsect.plot_grid(alpha=0.2)
    ax.set_ylim([0, 40])  # set y axis range to ignore low elevations
# -

# The `head` argument can be a 1D array or a 2D array matching the shape of the grid (i.e., `head.shape == (layer count, ncpl)`).

# +
line = lines[0]

for time in times[0:3]:
    head = np.array(hds.get_data(totim=time))
    head2 = np.hstack(head)

    fig = plt.figure(figsize=(25, 5))
    ax = fig.add_subplot(1, 3, 1)
    ax.set_title(f"Freyberg cross-section (t = {int(time)}, no head)")
    xsect = flopy.plot.PlotCrossSection(
        modelgrid=mfgrid, ax=ax, line={"line": line}, geographic_coords=True
    )
    cmap = xsect.plot_array(head2, masked_values=[-999.99], alpha=0.4)
    contours = xsect.contour_array(head2, levels=levels, alpha=1.0, colors="blue")
    xsect.plot_inactive(ibound=ibound, color_noflow=(0.8, 0.8, 0.8))
    xsect.plot_grid(alpha=0.2)
    ax.set_ylim([0, 40])  # set y axis range to ignore low elevations

    ax = fig.add_subplot(1, 3, 2)
    ax.set_title(
        f"Freyberg head cross-section (t = {int(time)}, head shape = {head.shape})"
    )
    xsect = flopy.plot.PlotCrossSection(
        modelgrid=mfgrid, ax=ax, line={"line": line}, geographic_coords=True
    )
    cmap = xsect.plot_array(head, masked_values=[-999.99], head=head, alpha=0.4)
    contours = xsect.contour_array(
        head, head=head, levels=levels, alpha=1.0, colors="blue"
    )
    xsect.plot_inactive(ibound=ibound, color_noflow=(0.8, 0.8, 0.8))
    xsect.plot_grid(alpha=0.2)
    ax.set_ylim([0, 40])

    ax = fig.add_subplot(1, 3, 3)
    ax.set_title(
        f"Freyberg head cross-section (t = {int(time)}, head shape = {head2.shape})"
    )
    xsect = flopy.plot.PlotCrossSection(
        modelgrid=mfgrid, ax=ax, line={"line": line}, geographic_coords=True
    )
    cmap = xsect.plot_array(head2, masked_values=[-999.99], head=head2, alpha=0.4)
    contours = xsect.contour_array(
        head2, head=head2, levels=levels, alpha=1.0, colors="blue"
    )
    xsect.plot_inactive(ibound=ibound, color_noflow=(0.8, 0.8, 0.8))
    xsect.plot_grid(alpha=0.2)
    ax.set_ylim([0, 40])
# -

try:
    # ignore PermissionError on Windows
    temp_dir.cleanup()
except:
    pass
